In [ ]:
import h5py
import numpy as np
from scipy.optimize import curve_fit, leastsq, fsolve
from scipy.constants import h, hbar
import matplotlib.pyplot as plt

data_file_path = '../data/'
folder_name = 'LKIPA_2/'
h5_filename = 'LKIPA_2.hdf5'
f = h5py.File(data_file_path + folder_name + h5_filename)

test_name = list(f.keys())[1]
test_runs = list(f[test_name].keys())
test_run = test_runs[12]

meas_num = 0

S_11_data = f[test_name][test_run][str(meas_num)]['Data'][()]
meas_freq = f[test_name][test_run][str(meas_num)]['Frequency'][()]
v_amp = f[test_name][test_run][str(meas_num)].attrs["v_amp"]
temp = f[test_name][test_run].attrs["T"]

if f[test_name][test_run].attrs["Script name"] == "sweep_cavity_vs_DC_bias_NCO_presto.py":
    DC_bias = True
    if f[test_name][test_run].attrs["Comment"] == "100KOhm":
        DC_resistance = 1e5
    elif f[test_name][test_run].attrs["Comment"] == "1MOhm":
        DC_resistance = 1e6
    else:
        raise Exception("DC resistance unknown")
else:
    DC_bias = False
    DC_resistance = 0

### Find group delay

##phi: phase shift, delay: group delay
def group_delay(freq, phi, delay):
    return phi - 2*np.pi * delay * freq

def correct_delay(S_11_data, meas_freq, delay):
    return S_11_data * np.exp(2*np.pi* 1j * meas_freq * delay)
delay = 75.5e-9

S_11_data_corr = correct_delay(S_11_data, meas_freq, delay)

print(f"Sample name: {test_name}")
print(f"Measurment time: {test_run}")
print(f"Temperature: {temp}")

from matplotlib import colormaps as cm

start_num = 0
num_curves = 9
cmap = cm['viridis']
legend = []

ReIm = False
if ReIm:
    fig, ax = plt.subplots(2, 2, figsize=(12, 10))
else:
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

for i in range(start_num, num_curves):
    meas_num = i

    S_11_data = f[test_name][test_run][str(meas_num)]['Data'][()]
    freq = f[test_name][test_run][str(meas_num)]['Frequency'][()]
    v_amp = f[test_name][test_run][str(meas_num)].attrs["v_amp"]

    legend.append(f"(i={i})={v_amp:2f}")

    S_11_data_corr = correct_delay(S_11_data, freq, delay)

    if DC_bias:
        S_11_data_corr_rel = S_11_data_corr
    else:
        S_11_data_corr_rel = S_11_data_corr/v_amp

    
    color = cmap((i - start_num) / (num_curves - start_num - 1))

    if ReIm:
        ax[0,0].plot(freq, 10*np.log10(np.abs(S_11_data_corr_rel)), color = color)
        ax[0,1].plot(freq, np.angle(S_11_data_corr_rel), color = color)

        ax[1,0].plot(freq, np.real(S_11_data_corr_rel), color = color)
        ax[1,1].plot(freq, np.imag(S_11_data_corr_rel), color = color)

        ax[0,0].set_title("Reflection Magnitude |S11| ()")
        ax[0,0].set_xlabel("Angular Frequency (Hz)")
        ax[0,0].set_ylabel("|S11|")

        ax[0,1].set_title("Reflection Phase ∠S11")
        ax[0,1].set_xlabel("Angular Frequency (Hz)")
        ax[0,1].set_ylabel("Phase (radians)")
        ax[0,1].grid(True, which='both')

        ax[1,0].set_title("Real part of S11")
        ax[1,0].set_xlabel("Angular Frequency (Hz)")
        ax[1,0].set_ylabel("Re{S11}")

        ax[1,1].set_title("Imaginary part of S11")
        ax[1,1].set_xlabel("Angular Frequency (Hz)")
        ax[1,1].set_ylabel("Im{S11}")
        ax[1,1].grid(True, which='both')
    else:
        ax[0].plot(freq, 10*np.log10(np.abs(S_11_data_corr_rel)), color = color)
        ax[1].plot(freq, np.angle(S_11_data_corr_rel), color = color)

        ax[0].set_title("Reflection Magnitude |S11| ()")
        ax[0].set_xlabel("Angular Frequency (Hz)")
        ax[0].set_ylabel("|S11|")

        ax[1].set_title("Reflection Phase ∠S11")
        ax[1].set_xlabel("Angular Frequency (Hz)")
        ax[1].set_ylabel("Phase (radians)")
        ax[1].grid(True, which='both')

ax[0].legend(legend, title = "v_amp")
fig.savefig("fig_imag0")


KeyError: "Unable to synchronously open attribute (can't locate attribute: 'v_amp')"